In [1]:
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies lightautoml==0.3.8
!pip install --no-index -U --find-links=/kaggle/input/lightautoml-038-dependecies pandas==2.0.3

Looking in links: /kaggle/input/lightautoml-038-dependecies
Processing /kaggle/input/lightautoml-038-dependecies/lightautoml-0.3.8-py3-none-any.whl
Processing /kaggle/input/lightautoml-038-dependecies/AutoWoE-1.3.2-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/cmaes-0.10.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/joblib-1.2.0-py3-none-any.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/json2html-1.3.0.tar.gz (from lightautoml==0.3.8)
  Preparing metadata (setup.py) ... done
Processing /kaggle/input/lightautoml-038-dependecies/lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (from lightautoml==0.3.8)
Processing /kaggle/input/lightautoml-038-dependecies/poetry_core-1.8.1-py3-none-any.whl (from lightautoml

## Set Global Configuration Options

In [11]:
class CONFIG:
    '''
    > General Options
    '''
    # global seed
    seed = 42
    # the number of samples to use for testing purposes
    # if None, we use the full dataset
    samples_testing = None #None
    # max rows to display for pandas dataframes
    display_max_rows = 200
    # name of the response variate we are trying to predict
    response_variate = 'score'
    # minimum value for response variate
    min_possible_response_value = 0.5
    # maximum value for response variate
    max_possible_response_value = 6.0
    
    '''
    > Feature Engineering Options
    '''
    # whether to use pre feature engineered data or not
    use_pre_fe_data = True
    # fe data saved path
    pre_fe_data_filepath = '/kaggle/input/writing-quality-baseline-v2-train-data/feat_eng_train_feats.csv'
    
    '''
    > Preprocessing Options
    '''
    # number of folds to split the data for CV
    num_folds = 10
    
    '''
    > Modelling + Training Options
    '''
    # the names of the models to use
    # either a list of model names, or 'all', in which case all models are used
    model_names = 'all'
    # number of trials to use for early stopping
    num_trials_early_stopping = 50
    # model path for lightautoml
    lightautoml_model_path = '/kaggle/input/writing-quality-baseline-v2-lightautoml/denselight.model'
    # oof preds path for lightautoml
    lightautoml_oof_preds_path = '/kaggle/input/writing-quality-baseline-v2-lightautoml/denselight_oof_preds'
    
    '''
    > Post-Modelling Options
    '''
    # number of most important features to display
    # for feature importances plots
    num_features_to_display = 50

## Import Libraries

In [12]:
import warnings

import os
import gc
import re
import random
from collections import Counter, defaultdict
import pprint
import pickle
import time
import copy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.autonotebook import tqdm

# from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder, PowerTransformer, RobustScaler, FunctionTransformer
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

import lightgbm as lgb
import xgboost as xgb
import catboost as cb
import optuna

## Set Some Options

In [13]:
tqdm.pandas()
sns.set_style("whitegrid")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", CONFIG.display_max_rows)
warnings.simplefilter('ignore')

random.seed(CONFIG.seed)

## Load Data

In [14]:
%%time
INPUT_DIR = '/kaggle/input/linking-writing-processes-to-writing-quality'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')

CPU times: user 10.6 s, sys: 2.33 s, total: 13 s
Wall time: 13 s


## Subsample Data (If Specified)

In [15]:
if CONFIG.samples_testing is not None:
    ids = list(train_logs["id"].unique())
    sample_ids = random.sample(ids, CONFIG.samples_testing)
    train_logs = train_logs[train_logs["id"].isin(sample_ids)]

## Looking At Data

In [16]:
train_logs.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [17]:
train_scores.head()

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


In [18]:
test_logs.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0


## Feature Engineering

### Essay Constructor

In [20]:
class EssayConstructor:
    
    def processingInputs(self,currTextInput):
        # Where the essay content will be stored
        essayText = ""
        # Produces the essay
        for Input in currTextInput.values:
            # Input[0] = activity
            # Input[1] = cursor_position
            # Input[2] = text_change
            # Input[3] = id
            # If activity = Replace
            if Input[0] == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

            # If activity = Move...
            if "M" in Input[0]:
                # Gets rid of the "Move from to" text
                croppedTxt = Input[0][10:]              
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')              
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]              
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue                
                
            # If activity = input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        return essayText
            
            
    def getEssays(self,df):
        # Copy required columns
        textInputDf = copy.deepcopy(df[['id', 'activity', 'cursor_position', 'text_change']])
        # Get rid of text inputs that make no change
        textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']     
        # construct essay, fast 
        tqdm.pandas()
        essay=textInputDf.groupby('id')[['activity','cursor_position', 'text_change']].progress_apply(lambda x: self.processingInputs(x))      
        # to dataframe
        essayFrame=essay.to_frame().reset_index()
        essayFrame.columns=['id','essay']
        # Returns the essay series
        return essayFrame

## Preprocessor Class

In [21]:
# nth percentile function for agg
def percentile(n):
    def percentile_(x):
        return x.quantile(n/100)
    percentile_.__name__ = 'pct_{:02.0f}'.format(n)
    return percentile_

def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

class Preprocessor:
    def __init__(self, seed):
        self.seed = seed
        
        self.activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
        self.events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
              'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
        self.text_changes_dict = {
            'q': 'q', 
            ' ': 'space', 
            'NoChange': 'NoChange', 
            '.': 'full_stop', 
            ',': 'comma', 
            '\n': 'newline', 
            "'": 'single_quote', 
            '"': 'double_quote', 
            '-': 'dash', 
            '?': 'question_mark', 
            ';': 'semicolon', 
            '=': 'equals', 
            '/': 'slash', 
            '\\': 'double_backslash', 
            ':': 'colon'
        }
        self.punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                        '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
        self.gaps = [1, 2, 3, 5, 10, 20, 50, 70, 100]
        self.percentiles = [5, 10, 25, 50, 75, 90, 95]
        self.percentiles_cols = [percentile(n) for n in self.percentiles]
        self.aggregations = ['mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']
        self.idf = defaultdict(float)
        
        self.essay_constructor = EssayConstructor()
    
    def get_essay_aggregations(self, essay_df):
        cols_to_drop = ['essay']
        # Total essay length
        essay_df['essay_len'] = essay_df['essay'].apply(lambda x: len(x))
        essay_df = essay_df.drop(columns=cols_to_drop)
        return essay_df
    
    def split_essays_into_words(self, essay_df):
        essay_df['word'] = essay_df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
        essay_df = essay_df.explode('word')
        # Word length (number of characters in word)
        essay_df['word_len'] = essay_df['word'].apply(lambda x: len(x))
        essay_df = essay_df[essay_df['word_len'] != 0]
        return essay_df
    
    def compute_word_aggregations(self, word_df):
        word_agg_df = word_df[['id','word_len']].groupby(['id']).agg(self.aggregations)
        word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
        word_agg_df['id'] = word_agg_df.index
        # New features: computing the # of words whose length exceed word_l
        for word_l in [5, 6, 7, 8, 9, 10, 11, 12]:
            word_agg_df[f'word_len_ge_{word_l}_count'] = word_df[word_df['word_len'] >= word_l].groupby(['id']).count().iloc[:, 0]
            word_agg_df[f'word_len_ge_{word_l}_count'] = word_agg_df[f'word_len_ge_{word_l}_count'].fillna(0)
        word_agg_df = word_agg_df.reset_index(drop=True)
        return word_agg_df
    
    def split_essays_into_sentences(self, essay_df):
        essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
        essay_df = essay_df.explode('sent')
        essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
        # Number of characters in sentences
        essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
        # Number of words in sentences
        essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
        essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
        return essay_df

    def compute_sentence_aggregations(self, sent_df):
        sent_agg_df = sent_df[['id','sent_len','sent_word_count']].groupby(['id']).agg(self.aggregations)
        sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
        sent_agg_df['id'] = sent_agg_df.index
        # New features: computing the # of sentences whose (character) length exceed sent_l
        for sent_l in [50, 60, 75, 100]:
            sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_df[sent_df['sent_len'] >= sent_l].groupby(['id']).count().iloc[:, 0]
            sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_agg_df[f'sent_len_ge_{sent_l}_count'].fillna(0)
        sent_agg_df = sent_agg_df.reset_index(drop=True)
        return sent_agg_df

    def split_essays_into_paragraphs(self, essay_df):
        essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
        essay_df = essay_df.explode('paragraph')
        # Number of characters in paragraphs
        essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
        # Number of sentences in paragraphs
        essay_df['paragraph_sent_count'] = essay_df['paragraph'].apply(lambda x: len(x.split('\\.|\\?|\\!')))
        # Number of words in paragraphs
        essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
        essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
        return essay_df

    def compute_paragraph_aggregations(self, paragraph_df):
        paragraph_agg_df = paragraph_df[['id','paragraph_len', 'paragraph_sent_count', 'paragraph_word_count']].groupby(['id']).agg(self.aggregations)
        paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
        paragraph_agg_df['id'] = paragraph_agg_df.index
        paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
        return paragraph_agg_df
        
    def activity_counts(self, df):
        tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['activity'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.activities:
                di[k] = 0
            # make dictionary entry for "move from X to Y"
            di["move_to"] = 0
            
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
                else:
                    # we can do this because there are no missing values
                    di["move_to"] += v
            ret.append(di)
        
        ret = pd.DataFrame(ret)
        # using tfidf
        ret_tfidf = pd.DataFrame(ret)
        # returning counts as is
        ret_normal = pd.DataFrame(ret)
        
        tfidf_cols = [f'activity_{act}_tfidf_count' for act in ret.columns]
        normal_cols = [f'activity_{act}_normal_count' for act in ret.columns]
        
        ret_tfidf.columns = tfidf_cols
        ret_normal.columns = normal_cols
        
        '''
        Credit: https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features
        '''
        cnts = ret_tfidf.sum(1)

        for col in tfidf_cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret_tfidf[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret_tfidf[col] = 1 + np.log(ret_tfidf[col] / cnts)
            ret_tfidf[col] *= idf
        
        ret_agg = pd.concat([ret_tfidf, ret_normal], axis=1)
        return ret_agg

    def event_counts(self, df, colname):
        tmp_df = df.groupby('id').agg({colname: list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df[colname].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.events:
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
            
        ret = pd.DataFrame(ret)
        # using tfidf
        ret_tfidf = pd.DataFrame(ret)
        # returning counts as is
        ret_normal = pd.DataFrame(ret)
        
        tfidf_cols = [f'{colname}_{event}_tfidf_count' for event in ret.columns]
        normal_cols = [f'{colname}_{event}_normal_count' for event in ret.columns]
        
        ret_tfidf.columns = tfidf_cols
        ret_normal.columns = normal_cols
        
        '''
        Credit: https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features
        '''
        cnts = ret_tfidf.sum(1)

        for col in tfidf_cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret_tfidf[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret_tfidf[col] = 1 + np.log(ret_tfidf[col] / cnts)
            ret_tfidf[col] *= idf
        
        ret_agg = pd.concat([ret_tfidf, ret_normal], axis=1)
        return ret_agg

    def text_change_counts(self, df):
        tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['text_change'].values):
            items = list(Counter(li).items())
            di = dict()
            for k in self.text_changes_dict.keys():
                di[k] = 0
            for item in items:
                k, v = item[0], item[1]
                if k in di:
                    di[k] = v
            ret.append(di)
            
        ret = pd.DataFrame(ret)
        # using tfidf
        ret_tfidf = pd.DataFrame(ret)
        # returning counts as is
        ret_normal = pd.DataFrame(ret)
        
        tfidf_cols = [f'text_change_{self.text_changes_dict[txt_change]}_tfidf_count' for txt_change in ret.columns]
        normal_cols = [f'text_change_{self.text_changes_dict[txt_change]}_normal_count' for txt_change in ret.columns]
        
        ret_tfidf.columns = tfidf_cols
        ret_normal.columns = normal_cols
        
        '''
        Credit: https://www.kaggle.com/code/olyatsimboy/towards-tf-idf-in-logs-features
        '''
        cnts = ret_tfidf.sum(1)

        for col in tfidf_cols:
            if col in self.idf.keys():
                idf = self.idf[col]
            else:
                idf = df.shape[0] / (ret_tfidf[col].sum() + 1)
                idf = np.log(idf)
                self.idf[col] = idf

            ret_tfidf[col] = 1 + np.log(ret_tfidf[col] / cnts)
            ret_tfidf[col] *= idf
        
        ret_agg = pd.concat([ret_tfidf, ret_normal], axis=1)
        return ret_agg
    
    def match_punctuations(self, df):
        tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
        ret = list()
        for li in tqdm(tmp_df['down_event'].values):
            cnt = 0
            items = list(Counter(li).items())
            for item in items:
                k, v = item[0], item[1]
                if k in self.punctuations:
                    cnt += v
            ret.append(cnt)
        ret = pd.DataFrame({'punct_cnt': ret})
        return ret

    # Credit: https://www.kaggle.com/code/abdullahmeda/enter-ing-the-timeseries-space-sec-3-new-aggs/notebook
    def make_space_features(self, df):
        df['up_time_lagged'] = df.groupby('id')['up_time'].shift(1).fillna(df['down_time'])
        df['time_diff'] = abs(df['down_time'] - df['up_time_lagged']) / 1000

        group = df.groupby('id')['time_diff']
        largest_lantency = group.max()
        smallest_lantency = group.min()
        median_lantency = group.median()
        initial_pause = df.groupby('id')['down_time'].first() / 1000
        pauses_half_sec = group.apply(lambda x: ((x > 0.5) & (x < 1)).sum())
        pauses_1_sec = group.apply(lambda x: ((x > 1) & (x < 1.5)).sum())
        pauses_1_half_sec = group.apply(lambda x: ((x > 1.5) & (x < 2)).sum())
        pauses_2_sec = group.apply(lambda x: ((x > 2) & (x < 3)).sum())
        pauses_3_sec = group.apply(lambda x: (x > 3).sum())
        
        result = pd.DataFrame({
            'id': df['id'].unique(),
            'largest_lantency': largest_lantency,
            'smallest_lantency': smallest_lantency,
            'median_lantency': median_lantency,
            'initial_pause': initial_pause,
            'pauses_half_sec': pauses_half_sec,
            'pauses_1_sec': pauses_1_sec,
            'pauses_1_half_sec': pauses_1_half_sec,
            'pauses_2_sec': pauses_2_sec,
            'pauses_3_sec': pauses_3_sec,
        }).reset_index(drop=True)
        return result
    
    def get_input_words(self, df):
        tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
        tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
        tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
        tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
        tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
        for percentile in self.percentiles:
            tmp_df[f'input_word_length_pct_{percentile}'] = tmp_df['text_change'].apply(lambda x: np.percentile([len(i) for i in x] if len(x) > 0 else 0, 
                                                                                                               percentile))
        tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
        tmp_df.drop(['text_change'], axis=1, inplace=True)
        return tmp_df
    
    def make_feats(self, df: pd.DataFrame, save_essays_path: str):
        
        print("Starting to engineer features")
        
        # initialize features dataframe
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        # get essay feats
        print("Getting essays")
        essay_df = self.essay_constructor.getEssays(df)
        essay_df.to_csv(save_essays_path, index=False)

        print("Getting essay aggregations data")
        essay_agg_df = self.get_essay_aggregations(essay_df)
        feats = feats.merge(essay_agg_df, on='id', how='left')

        print("Getting essay word aggregations data")
        word_df = self.split_essays_into_words(essay_df)
        word_agg_df = self.compute_word_aggregations(word_df)
        feats = feats.merge(word_agg_df, on='id', how='left')

        print("Getting essay sentence aggregations data")
        sent_df = self.split_essays_into_sentences(essay_df)
        sent_agg_df = self.compute_sentence_aggregations(sent_df)
        feats = feats.merge(sent_agg_df, on='id', how='left')

        print("Getting essay paragraph aggregations data")
        paragraph_df = self.split_essays_into_paragraphs(essay_df)
        paragraph_agg_df = self.compute_paragraph_aggregations(paragraph_df)
        feats = feats.merge(paragraph_agg_df, on='id', how='left')
        
        # engineer counts data
        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        # space features
        print("Engineering space-related data")
        tmp_df = self.make_space_features(df)
        feats = feats.merge(tmp_df, on='id', how='left')
        
        # get shifted features
        # time shift
        print("Engineering time data")
        for gap in self.gaps:
            print(f"> for gap {gap}")
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)
        
        # cursor position shift
        print("Engineering cursor position data - gaps")
        for gap in self.gaps: 
            print(f"> for gap {gap}")
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)
        
        # word count shift
        print("Engineering word count data - gaps")
        for gap in self.gaps: 
            print(f"> for gap {gap}")
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)
        
        # get aggregate statistical features
        print("Engineering statistical summaries for features")
        # [(feature name, [ stat summaries to add ])]
        percentiles_cols = [percentile(n) for n in self.percentiles]
        feats_stat = [
            ('event_id', ['max']),
            ('up_time', ['first', 'last', 'max']),
            ('down_time', ['first', 'last', 'max']),
            ('action_time', ['max', 'mean', 'std', 'sem', 'skew', pd.DataFrame.kurt ] + self.percentiles_cols),
            ('activity', ['nunique']),
            ('down_event', [ 'nunique']),
            ('up_event', [ 'nunique']),
            ('text_change', [ 'nunique']),
            ('cursor_position', ['max']),
            ('word_count', ['max'])] 

        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['first','last', 'max', 'min', 'mean', 'std', 'sem', 'skew', pd.DataFrame.kurt]+ percentiles_cols),
                (f'cursor_position_change{gap}', ['first','last','max', 'mean', 'std','sem', 'skew', pd.DataFrame.kurt]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'sum', 'sem', 'skew', pd.DataFrame.kurt] + percentiles_cols),
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                    
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left') 

        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')
        
        # compare feats
        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
        feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_mean'] / feats['up_time_max']
        
        print("Done!")
        return feats

## Run Preprocessor

In [22]:
if CONFIG.use_pre_fe_data:
    print("-"*25)
    print("Loading pre-engineered features for training data")
    print("-"*25)
    train_feats = pd.read_csv(CONFIG.pre_fe_data_filepath)
else:
    preprocessor_train = Preprocessor(seed=CONFIG.seed)
    print("-"*25)
    print("Engineering features for training data")
    print("-"*25)
    train_feats = preprocessor_train.make_feats(train_logs, save_essays_path = 'train_essays.csv')
    del preprocessor_train
    gc.collect()
    
print()
print("-"*25)
print("Engineering features for test data")
print("-"*25)
preprocessor_test = Preprocessor(seed=CONFIG.seed)
test_feats = preprocessor_test.make_feats(test_logs, save_essays_path='test_essays.csv')
del preprocessor_test
gc.collect()

-------------------------
Loading pre-engineered features for training data
-------------------------

-------------------------
Engineering features for test data
-------------------------
Starting to engineer features
Getting essays


  0%|          | 0/3 [00:00<?, ?it/s]

Getting essay aggregations data
Getting essay word aggregations data
Getting essay sentence aggregations data
Getting essay paragraph aggregations data
Engineering activity counts data


  0%|          | 0/3 [00:00<?, ?it/s]

Engineering event counts data


...

  0%|          | 0/3 [00:00<?, ?it/s]

Engineering space-related data
Engineering time data
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 70
> for gap 100
Engineering cursor position data - gaps
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 70
> for gap 100
Engineering word count data - gaps
> for gap 1
> for gap 2
> for gap 3
> for gap 5
> for gap 10
> for gap 20
> for gap 50
> for gap 70
> for gap 100
Engineering statistical summaries for features


  0%|          | 0/37 [00:00<?, ?it/s]

Engineering input words data
Engineering ratios data
Done!


152

In [23]:
train_feats.to_csv("feat_eng_train_feats.csv", index=False)

In [24]:
print(f"Shape of training data : {train_feats.shape}" )
print(f"Shape of test data : {test_feats.shape}" )

Shape of training data : (2471, 591)
Shape of test data : (3, 591)


In [25]:
assert train_feats.shape[1] == test_feats.shape[1], "Train and test data must have same number of features."

In [26]:
train_feats.head()

,id,essay_len,word_len_mean,word_len_std,word_len_min,word_len_max,word_len_first,word_len_last,word_len_sem,word_len_q1,word_len_median,word_len_q3,word_len_skew,word_len_kurt,word_len_sum,word_len_ge_5_count,word_len_ge_6_count,word_len_ge_7_count,word_len_ge_8_count,word_len_ge_9_count,word_len_ge_10_count,word_len_ge_11_count,word_len_ge_12_count,sent_len_mean,sent_len_std,sent_len_min,sent_len_max,sent_len_first,sent_len_last,sent_len_sem,sent_len_q1,sent_len_median,sent_len_q3,sent_len_skew,sent_len_kurt,sent_len_sum,sent_word_count_mean,sent_word_count_std,sent_word_count_min,sent_word_count_max,sent_word_count_first,sent_word_count_last,sent_word_count_sem,sent_word_count_q1,sent_word_count_median,sent_word_count_q3,sent_word_count_skew,sent_word_count_kurt,sent_word_count_sum,sent_len_ge_50_count,sent_len_ge_60_count,sent_len_ge_75_count,sent_len_ge_100_count,paragraph_len_mean,paragraph_len_std,paragraph_len_min,paragraph_len_max,paragraph_len_first,paragraph_len_last,paragraph_len_sem,paragraph_len_q1,paragraph_len_median,paragraph_len_q3,paragraph_len_skew,paragraph_len_kurt,paragraph_len_sum,paragraph_sent_count_mean,paragraph_sent_count_std,paragraph_sent_count_min,paragraph_sent_count_max,paragraph_sent_count_first,paragraph_sent_count_last,paragraph_sent_count_sem,paragraph_sent_count_q1,paragraph_sent_count_median,paragraph_sent_count_q3,paragraph_sent_count_skew,paragraph_sent_count_kurt,paragraph_sent_count_sum,paragraph_word_count_mean,paragraph_word_count_std,paragraph_word_count_min,paragraph_word_count_max,paragraph_word_count_first,paragraph_word_count_last,paragraph_word_count_sem,paragraph_word_count_q1,paragraph_word_count_median,paragraph_word_count_q3,paragraph_word_count_skew,paragraph_word_count_kurt,paragraph_word_count_sum,activity_Input_tfidf_count,activity_Remove/Cut_tfidf_count,activity_Nonproduction_tfidf_count,activity_Replace_tfidf_count,activity_Paste_tfidf_count,activity_move_to_tfidf_count,activity_Input_normal_count,activity_Remove/Cut_normal_count,activity_Nonproduction_normal_count,activity_Replace_normal_count,activity_Paste_normal_count,activity_move_to_normal_count,down_event_q_tfidf_count,down_event_Space_tfidf_count,down_event_Backspace_tfidf_count,down_event_Shift_tfidf_count,down_event_ArrowRight_tfidf_count,down_event_Leftclick_tfidf_count,down_event_ArrowLeft_tfidf_count,down_event_._tfidf_count,"down_event_,_tfidf_count",down_event_ArrowDown_tfidf_count,down_event_ArrowUp_tfidf_count,down_event_Enter_tfidf_count,down_event_CapsLock_tfidf_count,down_event_'_tfidf_count,down_event_Delete_tfidf_count,down_event_Unidentified_tfidf_count,down_event_q_normal_count,down_event_Space_normal_count,down_event_Backspace_normal_count,down_event_Shift_normal_count,down_event_ArrowRight_normal_count,down_event_Leftclick_normal_count,down_event_ArrowLeft_normal_count,down_event_._normal_count,"down_event_,_normal_count",down_event_ArrowDown_normal_count,down_event_ArrowUp_normal_count,down_event_Enter_normal_count,down_event_CapsLock_normal_count,down_event_'_normal_count,down_event_Delete_normal_count,down_event_Unidentified_normal_count,up_event_q_tfidf_count,up_event_Space_tfidf_count,up_event_Backspace_tfidf_count,up_event_Shift_tfidf_count,up_event_ArrowRight_tfidf_count,up_event_Leftclick_tfidf_count,up_event_ArrowLeft_tfidf_count,up_event_._tfidf_count,"up_event_,_tfidf_count",up_event_ArrowDown_tfidf_count,up_event_ArrowUp_tfidf_count,up_event_Enter_tfidf_count,up_event_CapsLock_tfidf_count,up_event_'_tfidf_count,up_event_Delete_tfidf_count,up_event_Unidentified_tfidf_count,up_event_q_normal_count,up_event_Space_normal_count,up_event_Backspace_normal_count,up_event_Shift_normal_count,up_event_ArrowRight_normal_count,up_event_Leftclick_normal_count,up_event_ArrowLeft_normal_count,up_event_._normal_count,"up_event_,_normal_count",up_event_ArrowDown_normal_count,up_event_ArrowUp_normal_count,up_event_Enter_normal_count,up_event_CapsLock_normal_count,up_event_'_normal_count,up_event_D

In [27]:
test_feats.head()

,id,essay_len,word_len_mean,word_len_std,word_len_min,word_len_max,word_len_first,word_len_last,word_len_sem,word_len_q1,word_len_median,word_len_q3,word_len_skew,word_len_kurt,word_len_sum,word_len_ge_5_count,word_len_ge_6_count,word_len_ge_7_count,word_len_ge_8_count,word_len_ge_9_count,word_len_ge_10_count,word_len_ge_11_count,word_len_ge_12_count,sent_len_mean,sent_len_std,sent_len_min,sent_len_max,sent_len_first,sent_len_last,sent_len_sem,sent_len_q1,sent_len_median,sent_len_q3,sent_len_skew,sent_len_kurt,sent_len_sum,sent_word_count_mean,sent_word_count_std,sent_word_count_min,sent_word_count_max,sent_word_count_first,sent_word_count_last,sent_word_count_sem,sent_word_count_q1,sent_word_count_median,sent_word_count_q3,sent_word_count_skew,sent_word_count_kurt,sent_word_count_sum,sent_len_ge_50_count,sent_len_ge_60_count,sent_len_ge_75_count,sent_len_ge_100_count,paragraph_len_mean,paragraph_len_std,paragraph_len_min,paragraph_len_max,paragraph_len_first,paragraph_len_last,paragraph_len_sem,paragraph_len_q1,paragraph_len_median,paragraph_len_q3,paragraph_len_skew,paragraph_len_kurt,paragraph_len_sum,paragraph_sent_count_mean,paragraph_sent_count_std,paragraph_sent_count_min,paragraph_sent_count_max,paragraph_sent_count_first,paragraph_sent_count_last,paragraph_sent_count_sem,paragraph_sent_count_q1,paragraph_sent_count_median,paragraph_sent_count_q3,paragraph_sent_count_skew,paragraph_sent_count_kurt,paragraph_sent_count_sum,paragraph_word_count_mean,paragraph_word_count_std,paragraph_word_count_min,paragraph_word_count_max,paragraph_word_count_first,paragraph_word_count_last,paragraph_word_count_sem,paragraph_word_count_q1,paragraph_word_count_median,paragraph_word_count_q3,paragraph_word_count_skew,paragraph_word_count_kurt,paragraph_word_count_sum,activity_Input_tfidf_count,activity_Remove/Cut_tfidf_count,activity_Nonproduction_tfidf_count,activity_Replace_tfidf_count,activity_Paste_tfidf_count,activity_move_to_tfidf_count,activity_Input_normal_count,activity_Remove/Cut_normal_count,activity_Nonproduction_normal_count,activity_Replace_normal_count,activity_Paste_normal_count,activity_move_to_normal_count,down_event_q_tfidf_count,down_event_Space_tfidf_count,down_event_Backspace_tfidf_count,down_event_Shift_tfidf_count,down_event_ArrowRight_tfidf_count,down_event_Leftclick_tfidf_count,down_event_ArrowLeft_tfidf_count,down_event_._tfidf_count,"down_event_,_tfidf_count",down_event_ArrowDown_tfidf_count,down_event_ArrowUp_tfidf_count,down_event_Enter_tfidf_count,down_event_CapsLock_tfidf_count,down_event_'_tfidf_count,down_event_Delete_tfidf_count,down_event_Unidentified_tfidf_count,down_event_q_normal_count,down_event_Space_normal_count,down_event_Backspace_normal_count,down_event_Shift_normal_count,down_event_ArrowRight_normal_count,down_event_Leftclick_normal_count,down_event_ArrowLeft_normal_count,down_event_._normal_count,"down_event_,_normal_count",down_event_ArrowDown_normal_count,down_event_ArrowUp_normal_count,down_event_Enter_normal_count,down_event_CapsLock_normal_count,down_event_'_normal_count,down_event_Delete_normal_count,down_event_Unidentified_normal_count,up_event_q_tfidf_count,up_event_Space_tfidf_count,up_event_Backspace_tfidf_count,up_event_Shift_tfidf_count,up_event_ArrowRight_tfidf_count,up_event_Leftclick_tfidf_count,up_event_ArrowLeft_tfidf_count,up_event_._tfidf_count,"up_event_,_tfidf_count",up_event_ArrowDown_tfidf_count,up_event_ArrowUp_tfidf_count,up_event_Enter_tfidf_count,up_event_CapsLock_tfidf_count,up_event_'_tfidf_count,up_event_Delete_tfidf_count,up_event_Unidentified_tfidf_count,up_event_q_normal_count,up_event_Space_normal_count,up_event_Backspace_normal_count,up_event_Shift_normal_count,up_event_ArrowRight_normal_count,up_event_Leftclick_normal_count,up_event_ArrowLeft_normal_count,up_event_._normal_count,"up_event_,_normal_count",up_event_ArrowDown_normal_count,up_event_ArrowUp_normal_count,up_event_Enter_normal_count,up_event_CapsLock_normal_count,up_event_'_normal_count,up_event_D

In [28]:
train_feats = train_feats.merge(train_scores, on='id', how='left')

## Split Data Into Folds

In [29]:
kfold = KFold(n_splits=CONFIG.num_folds, shuffle=True, random_state=CONFIG.seed)

for fold, (_, val_idx) in enumerate(kfold.split(train_feats)):
    train_feats.loc[val_idx, "fold"] = fold

## Modeling

In [30]:
class WritingQualityModel:
    
    def __init__(self, model_name: str, params: dict):
        self.model_name = model_name
        self.model = self.create_model(model_name, params)
        
    def make_pipeline(self, model: str):
        return Pipeline([
            ('remove_infs', FunctionTransformer(lambda x: np.nan_to_num(x, nan=np.nan, posinf=0, neginf=0))),
            ('imputer', SimpleImputer(strategy='mean')),
            ('nomalizer', FunctionTransformer(lambda x: np.log1p(np.abs(x)))),
            ('scaler', RobustScaler()),
            ('model', model)
        ])
    
    def create_model(self, model_name: str, params: dict):
        model = None
        if 'lgbm' in model_name:
            model = lgb.LGBMRegressor(**params)
        elif 'cat' in model_name:
            model = cb.CatBoostRegressor(**params)
        elif 'rfr' in model_name:
            model = RandomForestRegressor(**params)
        elif 'lasso' in model_name:
            model = self.make_pipeline(Lasso(**params))
        return model
    
    def train(self, X_train, Y_train, X_val, Y_val):
        if any(x in self.model_name for x in ['lgbm']):
            early_stopping_callback = lgb.early_stopping(CONFIG.num_trials_early_stopping, first_metric_only=True, verbose=False)
            
            self.model.fit(X_train, Y_train,
                          eval_set=[(X_val, Y_val)],
                          eval_metric='rmse', verbose=0,
                          callbacks=[early_stopping_callback])
        
        elif any(x in self.model_name for x in ['cat']):
            self.model.fit(X_train, Y_train,
                          eval_set=[(X_val, Y_val)],
                          verbose=0,
                          early_stopping_rounds=CONFIG.num_trials_early_stopping)
        else:
            X_train = np.nan_to_num(X_train, posinf=-1, neginf=-1)
            self.model.fit(X_train, Y_train)
            
        return self.model
    
    def validate(self, X_val, Y_val):
        if any(x in self.model_name for x in ['rfr', 'ridge', 'lasso']):
            X_val = np.nan_to_num(X_val, posinf=-1, neginf=-1)
            
        pred = self.model.predict(X_val)
        score = mean_squared_error(pred, Y_val, squared=False)
        return pred, score
    
    def predict(self, X_test):
        if any(x in self.model_name for x in ['rfr', 'ridge', 'lasso']):
            X_test = np.nan_to_num(X_test, posinf=-1, neginf=-1)
            
        return self.model.predict(X_test)

## (3) LightGBM train and predic

In [ ]:
OOF_PREDS = np.zeros((len(train_feats), 2))
TEST_PREDS = np.zeros((len(test_feats), 2))

In [ ]:
# Code comes from here: https://www.kaggle.com/code/abdullahmeda/enter-ing-the-timeseries-space-sec-3-new-aggs

models_dict = {}
scores = []

test_predict_list = []
best_params = {'reg_alpha': 0.007678095440286993, 
               'reg_lambda': 0.34230534302168353, 
               'colsample_bytree': 0.627061253588415, 
               'subsample': 0.854942238828458, 
               'learning_rate': 0.038697981947473245, 
               'num_leaves': 22, 
               'max_depth': 37, 
               'min_child_samples': 18,
               'n_jobs':4
              }

for i in range(5): 
    kf = model_selection.KFold(n_splits=10, random_state=42 + i, shuffle=True)
    oof_valid_preds = np.zeros(train_feats.shape[0])
    X_test = test_feats[train_cols]
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train_feats)):
        
        X_train, y_train = train_feats.iloc[train_idx][train_cols], train_feats.iloc[train_idx][target_col]
        X_valid, y_valid = train_feats.iloc[valid_idx][train_cols], train_feats.iloc[valid_idx][target_col]
        params = {
            "objective": "regression",
            "metric": "rmse",
            'random_state': 42,
            "n_estimators" : 12001,
            "verbosity": -1,
            **best_params
        }
        model = lgb.LGBMRegressor(**params)
        early_stopping_callback = lgb.early_stopping(200, first_metric_only=True, verbose=False)
        
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],  
                  callbacks=[early_stopping_callback],
        )
        valid_predict = model.predict(X_valid)
        oof_valid_preds[valid_idx] = valid_predict
        OOF_PREDS[valid_idx, 0] += valid_predict / 5
        test_predict = model.predict(X_test)
        TEST_PREDS[:, 0] += test_predict / 5 / 10
        test_predict_list.append(test_predict)
        score = metrics.mean_squared_error(y_valid, valid_predict, squared=False)
        models_dict[f'{fold}_{i}'] = model

    oof_score = metrics.mean_squared_error(train_feats[target_col], oof_valid_preds, squared=False)
    scores.append(oof_score)

In [ ]:
print('OOF metric LGBM = {:.5f}'.format(metrics.mean_squared_error(train_feats[target_col], OOF_PREDS[:, 0], squared=False)))

## (4) LightAutoML NN (DenseLight) prediction

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import joblib

In [ ]:
for i in range(3):
    oof_pred, automl = joblib.load('/kaggle/input/linkinglamamodels/oof_and_lama_denselight_{}.pkl'.format(i))
    OOF_PREDS[:, 1] += oof_pred / 3
    TEST_PREDS[:, 1] += automl.predict(test_feats[train_cols]).data[:, 0] / 3

In [ ]:
print('OOF metric LightAutoML_NN = {:.5f}'.format(metrics.mean_squared_error(train_feats[target_col], OOF_PREDS[:, 1], squared=False)))

## (5) Blending

In [ ]:
best_sc = 1
for w in np.arange(0, 1.01, 0.001):
    sc = metrics.mean_squared_error(train_feats[target_col], 
                                    w * OOF_PREDS[:, 0] + (1-w) * OOF_PREDS[:, 1], 
                                    squared=False)
    if sc < best_sc:
        best_sc = sc
        best_w = w
        
print('Composition OOF score = {:.5f}'.format(best_sc))
print('Composition best W = {:.3f}'.format(best_w))

## (6) Submission creation

In [ ]:
W = [best_w, 1 - best_w]
test_preds = TEST_PREDS[:, 0] * W[0] + TEST_PREDS[:, 1] * W[1]
test_preds

In [ ]:
test_feats['score'] = test_preds
test_feats[['id', 'score']].to_csv("submission.csv", index=False)

In [ ]:
test_feats[['id', 'score']]